In [2]:
# Import
from ckanapi import RemoteCKAN
import pandas as pd
import os
import json
from string import Template
import ssl
import wget

# List **Groups** from CKAN repository

In [4]:
ckanUrl = 'http://dados.tre-rn.jus.br:5000'
ckan = RemoteCKAN(ckanUrl)
group_data = ckan.action.group_list(all_fields=True)
group_df = pd.DataFrame(group_data)
group_df

display_name description  \
0             Administração               
1                  Eleições               
2                Financeiro               
3  Governança Institucional               
4                   Pessoas               

                                   image_display_url  package_count  \
0  https://dados.tre-rn.jus.br/uploads/group/2021...              1   
1  https://dados.tre-rn.jus.br/uploads/group/2021...              1   
2  https://dados.tre-rn.jus.br/uploads/group/2021...              4   
3  https://dados.tre-rn.jus.br/uploads/group/2022...              3   
4  https://dados.tre-rn.jus.br/uploads/group/2021...              9   

                      created                      name  is_organization  \
0  2021-10-21T18:07:18.114516             administracao            False   
1  2021-10-21T18:06:58.286149                  eleicoes            False   
2  2021-10-27T11:50:05.813717                financeiro            False   
3  2022-04-20T18:51:43.638049  governanca-institucional            False   
4  2021-10-21T18:06:26.242963                   pessoas            False   

    state                                          image_url   type  \
0  active          2021-11-23-163127.672442ADMINISTRACAO.png  group   
1  active               2021-11-23-163145.779717ELEICOES.png  group   
2  active             2021-11-23-163206.448436FINANCEIRO.png  group   
3  active  2022-04-26-185127.9999712022-04-18-175514.7403...  group   
4  active                2021-11-23-163232.282292PESSOAS.png  group   

                      title  num_followers  \
0             Administração              0   
1                  Eleições              0   
2                Financeiro              0   
3  Governança Institucional              0   
4                   Pessoas              0   

                                     id approval_status  
0  0004eb42-850d-4191-8df9-6bf2d41ca053        approved  
1  553c739e-7af5-4ec0-aa2d-35b3b2c933b1        approved  
2  b4efa234-540a-4761-9e80-a8553f22bf9e        approved  
3  b4e092f5-8673-404a-97f5-7c60a02d8aa3        approved  
4  bbfca657-29e3-4abb-8054-0cb955c54157        approved

# List **Organizations** from CKAN repository

In [5]:
organizarion_data = ckan.action.organization_list(all_fields=True)
organizarion_df = pd.DataFrame(organizarion_data)
organizarion_df

display_name                                        description  \
0              SAOF  Secretaria de Administração, Orçamento e Finan...   
1               SGP          Secretaria de Gestão de Pessoas do TRE-RN   
2              STIE  Secretaria de Tecnologia da Informação e Eleiç...   
3  Zonas Eleitorais                         Zonas Eleitorais do TRE-RN   

                                   image_display_url  package_count  \
0  https://dados.tre-rn.jus.br/uploads/group/2021...              1   
1  https://dados.tre-rn.jus.br/uploads/group/2021...             16   
2  https://dados.tre-rn.jus.br/uploads/group/2021...              1   
3  https://dados.tre-rn.jus.br/uploads/group/2021...              2   

                      created              name  is_organization   state  \
0  2021-10-20T00:05:51.022583              saof             True  active   
1  2021-10-19T22:24:16.791726               sgp             True  active   
2  2021-10-21T18:06:48.519826             stiee             True  active   
3  2021-10-21T17:57:17.647337  zonas-eleitorais             True  active   

                                      image_url          type  \
0              2021-11-11-164101.347965SAOF.jpg  organization   
1               2021-11-11-164005.789030SGP.jpg  organization   
2              2021-11-11-164125.423528STIE.jpg  organization   
3  2021-11-11-164141.588491ZONAS-ELEITORAIS.jpg  organization   

              title  num_followers                                    id  \
0              SAOF              0  50d94c7a-3e6b-4371-9e94-2ca522884379   
1               SGP              0  daeb2684-dc9b-4d3a-9b77-c33c67a20767   
2              STIE              0  6e0811a6-e41f-4dbb-b754-9a66bf55c0ab   
3  Zonas Eleitorais              0  58d1e576-6c84-47e4-a8ca-be998f2642dd   

  approval_status  
0        approved  
1        approved  
2        approved  
3        approved

# Functions

In [6]:
# Create Json Config
# One json for each package

# Tempalate for package
package_template = Template('''{
    "name": "$name",
    "notes": "$notes",
    "private": "$private",
    "author": "$author",
    "author_email": "$author_email",
    "owner_org": "$owner_org",
    "version": $version,
    "tags": $tags,
    "id": "$id",
    "title": "$title"
}''')

# Template for resources
resource_template = Template('''{
    "author": "$author",
    "description": "$description",
    "package_id": "$package_id",
}''')

# package_data_with_resources is a list with dictionaries
package_data_with_resources = ckan.action.current_package_list_with_resources(limit=len(ckan.action.package_list()))


def template_to_json(template, item) -> str:
    """
        Get information from resource or package dictionaries, combine them
        at a template and return a string "JSON"

    Args:
        template (Template): String template for filtration of the data
        item (dict): dictionary with information about package or resource

    Returns:
        str: String JSON with the information of the package or resource
    """
    
    # Extract information from package each dictionary
    return template.safe_substitute(item).replace("'", '"').replace(': ,', ': "",').replace('None', '"None"')
 

# Process everything

1. Create directories for packages
2. Save a JSON config for package 
3. Create directories for resources
4. Download CSV/txt for each resource
5. Save a JSON config for resources

In [7]:
# Create package directories
path = "dump/"
_ = list(map(lambda package_name: os.makedirs(path + package_name['name'],  exist_ok=True), package_data_with_resources))

# Create resource subdirectories
ssl._create_default_https_context = ssl._create_unverified_context
for pkg in package_data_with_resources:
    # Iterate over packages
    dir_name = f'{path}{pkg["name"]}/'
    
    # Create JSON from string template
    package_json = json.loads(template_to_json(package_template, pkg), strict=False)
    # Save JSON in package directory
    save_file = open(f'{dir_name}{pkg["name"]}.json', 'w')
    save_file.write(json.dumps(package_json, indent=4))
    save_file.close()
        
    for rsc in pkg['resources']:
        # Iterate over resources
        # Get resources name
        rsc_dir_name = (rsc['url'].split('/', -1)[-1].replace('.csv', ''))
        
        # Concatenate directory name with resource name
        final_path = dir_name + rsc_dir_name 
        # Crate subdirectory
        os.makedirs(final_path, exist_ok=True)
        if rsc['url'] != '':
            # Download resource (CSV/TXT)
            wget.download(rsc['url'], out = f'{final_path}/')
            
            # Save resource Json config file
            save_file = open(f'{final_path}/{rsc_dir_name}.json', 'w')
            save_file.write(json.dumps(package_json, indent=4))
            save_file.close()
        

